pytorch는 두가지의 주요한 특징을 가짐
* Numpy와 유사하지만, GPU상에서 실행 가능한 N차원 Tensor
* 신경망을 구성하고 학습하는 과정에서의 자동미분

#### Numpy
Numpy는 과학적 분야의 연산을 위한 포괄적인 프레임워크   
Numpy는 연산그래프, 딥러닝, 변화도에 대해 알지 못하지만 연산만으로 순전파, 역전파 구현 가능

In [1]:
# -*- coding: utf-8 -*-
import numpy as np


# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 무작위로 가중치 초기화
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out) 

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    # 손실(loss) 계산
    loss = np.square(y_pred - y).sum()
    if t % 50 == 49:
        print(t + 1, loss)
    
    # 손실에 따른 w1, w2의 변화도 계산 및 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    # 가중치 갱신
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

50 12457.280273555938
100 372.5380233527984
150 17.574095542503116
200 0.9732836832049194
250 0.058413305868391996
300 0.003691982048735353
350 0.00024307173253735223
400 1.6582504644868267e-05
450 1.1681224584843756e-06
500 8.468383779353074e-08


#### Pytorch Tensor
현대의 심층 심경망에서 GPU는 50배 또는 그 이상의 속도 향상을 제공   
Pytorch Tensor는 개념적으로 Numpy 배열과 동일하며, N차원의 배열이고 다양한 함수를 제공   
Numpy 배열처럼 Tensor는 연산그래프, 딥러닝, 변화도는 알지 못하는 연산을 위한 포괄적 도구   
Numpy와는 달리 GPU 활용한 수치연산 가속화 가능

In [2]:
# -*- coding: utf-8 -*-
import torch


dtype = torch.float
device = torch.device("cpu")

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 무작위로 가중치 초기화
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y 계산
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # 손실(loss) 계산
    loss = (y_pred - y).pow(2).sum().item()
    if t % 50 == 49:
        print(t+1, loss)
    
    # 손실에 따른 w1, w2의 변화도 계산 및 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)
    
    # 경사하강법을 사용하여 가중치 갱신
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

50 11518.0849609375
100 454.9916687011719
150 30.12788963317871
200 2.3771533966064453
250 0.20440632104873657
300 0.01856987550854683
350 0.001993912737816572
400 0.0003746763104572892
450 0.00012145540677011013
500 5.565590981859714e-05


#### Pytorch: Tensor와 autograd
Pytorch의 autograd 패키지를 사용하면, 신경망의 순전파 단계에서 연산 그래프를 정의   
이 그래프의 노드(node)는 Tensor, 엣지(Edge)는 입력 Tensor로부터 출력 Tensor를 만듦   
이 그래프를 통해 역전파를 하게 되면 변화도를 쉽게 계산 할 수 있음
    
각 Tensor는 연산 그래프에서 노드로 표현   
만약, x가 x.requires_grad=True인 Tensor면 x.grad는 어떤 스칼라 값에 대한 x의 변화도를 갖는 또 다른 Tensor가 됨

In [3]:
# -*- coding: utf-8 -*-
import torch


dtype = torch.float
device = torch.device("cpu")

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성
# requires_grad=False로 설정하여 역전파 중에 이 Tensor들의 변화도 계산이 불필요함을 나타냄
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성
# requires_grad=True로 설정하여 역전파 중에 이 Tensor들의 변화도 계산이 필요함을 나타냄
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y값을 계산
    # 이는 Tensor를 사용한 순전파 단계와 동일하나, 역전파 단계를 별도 구성하지 않아도 되므로
    # 중간값들에 대한 참조(reference)를 갖고 있을 필요가 없음
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Tensor 연산을 사용하여 손실을 계산
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print(t+1, loss.item())
        
    # autograd를 사용하여 역전파 단계를 계산
    # requires_grad=True를 갖는 모든 Tensor에 대해 손실의 변화도를 계산
    # 이후, w1.grad와 w2.grad는 w1과 w2 각각에 대한 손실의 변화도를 갖는 Tensor가 됨
    loss.backward()
    
    # 경사하강법(gradient decent)를 사용하여 가중치를 수동으로 갱신
    # torch.no_grad()는 가중치들이 requires_grad=True이지만 autograd에서는 추적이 불필요 하므로 사용
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

50 15468.05859375
100 575.6167602539062
150 37.546470642089844
200 2.9725027084350586
250 0.25931107997894287
300 0.02403370663523674
350 0.0025631969328969717
400 0.0004514698521234095
450 0.00013966344704385847
500 6.214058521436527e-05


#### Pytorch: 새로운 autograd 함수 정의
내부적으로 autograd의 기본(primitive) 연산자는 실제로 Tensor를 조작하는 2개의 함수   
forward 함수는 압력한 Tensor로부터 출력 Tensor를 계산
backward 함수는 어떤 스칼라 값에 대한 출력 Tensor의 변화도를 전달받고, 동일한 스칼라 값에 대한 입력 Tensor의 변화도를 계산   
    
Pytorch에서는 torch.autograd.Funtion의 서브클래스를 정의하고 forward, backward 함수를 구현함으로써 사용자 정의 autograd 연산자를 정의 하는 것이 가능    
그 후 인스턴스(instance)를 생성하고 이를 함수처럼 호출하여 입력 데이터를 갖는 Tensor를 전달하는 식으로 새로운 autograd 연산자를 사용

In [4]:
# -*- coding: utf-8 -*-
import torch

class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        """
        순전파 단계에서는 입력을 갖는 Tensor를 받아 출력을 갖는 Tensor를 반환
        ctx는 컨텍스트 객체(context object)로 역전파 연산을 위한 정보 저장에 사용
        ctx.save_for_backward mathod를 사용하여 역전파 단계에서 사용할 객체도 저장 가능
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    @staticmethod
    def backward(ctx, grad_output):
        """
        역전파 단계에서는 출력에 대한 손실의 변화도를 갖는 Tensor를 입력받고, 손실을 계산
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0 ] = 0
        return grad_input


dtype = torch.float
device = torch.device("cpu")

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 무작위로 가중치 초기화
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 사용자 정의 Function을 적용하기 위해 Funtion.apply 메소드를 사용
    relu = MyReLU.apply
    
    # 순전파단계: Tensor 연상늘 사용하여 예상되는 y의 값 계산
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # 손실을 계산
    loss = (y_pred - y).pow(2).sum()
    if t % 50 == 49:
        print(t+1, loss.item())
        
    # autograd를 사용하여 역전파 계산
    loss.backward()
    
    # 경사하강법(gradient descent)를 사용하여 가중치 갱신
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # 가중치 갱신 후에는 수동으로 변화도를 0으로 초기화
        w1.grad.zero_()
        w2.grad.zero_()

50 15204.85546875
100 652.3815307617188
150 53.336082458496094
200 5.431611061096191
250 0.5915746688842773
300 0.06615796685218811
350 0.007709690369665623
400 0.0011498061940073967
450 0.0002816809865180403
500 0.000104223690868821


#### Pytorch: nn
규모가 큰 신경망에서는 autograd 그 자체만으로는 너무 낮은 수준(low-level) 일 수 있음    
nn 피키지는 신경망 계층(layer)들과 거의 동일한 Module의 집합을 정의   
    
Module은 입력 Tensor를 받고 출력 Tensor를 계산하는 한편, 학습 가능한 매개변수를 갖는 Tensor같은 내부 상태(Internal state)를 가짐    
nn 패키지는 또한 신경망을 학습시킬 때 주로 사용하는 유용한 손실 함수도 정의

In [5]:
# -*- coding: utf-8 -*-
import torch


# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉층 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn패키지를 시용하여 모델을 순차적 계층(sequence of layers)으로 정의
# nn.Sequential은 다른 Module들을 포한하는 Module
# Linear Module로 입력부터 출력을 계산하고, 내부 Tensor에 가중치 평듄을 저장
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)

# 또한 nn패키지는 널리 사용되는 손실 함수에 대한 정의도 포함
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y 값을 계산
    # __call__ 연산다를 덮어써(override) 함수를 호출
    # 입력 데이터의 Tensor를 Module에 전달하여 출력 데이터의 Tensor를 생성
    y_pred = model(x)
    
    # 손살을 계산
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print(t+1, loss.item())
    
    # 역전파 단계를 실행하기 전에 변화도를 0으로 초기화
    model.zero_grad()
    
    # 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도 계산
    # 내부적으로 각 Module의 매개변수는 requires_grad=True 일 때, Tensor내 저장
    # 따라서 이 호출은 모든 모델의 모든 학습 가능한 매개변수의 변화도를 계산함
    loss.backward()
    
    # 경사하강법(gradient descent)를 사용하여 가중치를 갱신
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

50 42.64419937133789
100 2.896549701690674
150 0.31992000341415405
200 0.043064504861831665
250 0.0065519725903868675
300 0.0010944054229184985
350 0.00019768897618632764
400 3.820127676590346e-05
450 7.824991371307988e-06
500 1.6879752138265758e-06


#### Pytorch: optim
autograd의 추적 기록을 피하기 위해 torch.no_grad() 또는 .data를 사용하는 삭으로 학습 가능한 매개변수를 갖는 Tensor를 직접 조작하여 모델의 가중치를 갱신하였음.   
이것은 SGD 같은 간단한 최적화 함수에는 부담이 없지만, 실제 신경망에서는 주로 AdaGrad, RMSProp, Adam 같은 정교한 Optimizer를 사용함   
   
Pytorch의 optim 패키지는 최적화 알고리즘에 대한 아이디어를 추상화 하고 일반적으로 사용되는 최적화 알고리즘의 구현체를 제공

In [6]:
# -*- coding: utf-8 -*-
import torch

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nn패키지를 사용하여 모델과 손실 함수를 정의
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H), 
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optim패키지를 사용하여 모델의 가중치를 갱신할 Optimizer를 정의 (Adam)
# Adam 생성자의 첫번째 인자는 어떤 Tensor가 갱신되어야 하는지 알려줌
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y값을 계산
    y_pred = model(x)
    
    # 손실을 계산하고 출력
    loss = loss_fn(y_pred, y)
    if t % 50 == 49:
        print(t+1, loss.item())
        
    # 역전파 단계 전에, Optimizer객체로 갱신할 변수들의 모든 변화도를 0으로 설정
    # 기본적으로 backward()를 호출할 때마다 변화도가 버퍼(buffer)에 누적됨
    # 이를 방지하기 위해 0으로 초기화
    optimizer.zero_grad()
    
    # 역전파 단계: 모델의 매개변수에 대한 손실의 변화도를 계산
    loss.backward()
    
    # Optimizer의 step 함수를 호출하면 매개변수가 갱신
    optimizer.step()

50 238.46392822265625
100 64.2444076538086
150 11.216009140014648
200 1.3561795949935913
250 0.1305735856294632
300 0.010254614986479282
350 0.0006217267364263535
400 3.1556432077195495e-05
450 1.6688805999365286e-06
500 9.897198083308467e-08


#### Pytorch: 사용자정의 nn.Module
때때로 기존 모듈의 구성(sequence)보다 더 복잡한 모델을 구성해야 할 때가 있음   
이럴때는, nn.Module의 서브클래스로 새 모듈을 정의하고, 입력 Tensor를 받아 다른 모듈 또는 Tensor의 autograd 연산을 사용하여 출력 Tensor를 만드는 forward를 정의

In [7]:
# -*- coding: utf-8 -*-
import torch


class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, D_out):
        """
        생성자에서 2개의 nn.Linear 모듈을 생성하고, 멤버 변수로 지정
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
    
    def forward(self, x):
        """
        순전파 함수에서는 입력 데이터의 Tensor를 받고 출력 데이터의 Tensor를 반환
        Tensor 상의 임의의 연산자 뿐 아니라 생성자에서 정의한 Module도 사용 가능
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
        

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 앞에서 정의한 클래스를 생성하여 모델을 구성
model = TwoLayerNet(D_in, D_out)

# 손실 함수와 Optimizer를 만듦
# SGD 생성자에 model.parameters()를 호출하면 모델의 멤버인 2개의 nn.Linear 모듈의
# 학습 가능한 매개변수들이 포함
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y값을 계산
    y_pred = model(x)
    
    # 손실을 계산
    loss = criterion(y_pred, y)
    if t % 50 == 49:
        print(t+1, loss.item())
    
    # 변화도를 0으로 만들고, 역전파를 수행하고, 가충치를 갱신함
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

50 1.5724143981933594
100 0.007448613177984953
150 4.29026476922445e-05
200 2.1645473680109717e-07
250 1.0345480028206566e-09
300 8.963182626664512e-12
350 2.604525536215041e-12
400 2.9177437375904614e-12
450 2.7558836289409694e-12
500 2.783892474184091e-12


#### Pytorch: 제어흐름(Control Flow) + 가중치 공유(Weight Sharing)
동적 그래프와 가중치 공유의 예로, 이상한 모데을 구현하여 예시로 삼음   
각 순전파 단계에서 많은 은닉 계층을 갖는 전결합된 ReLU 신경망이 무작위로 0~3 사이의 숫자를 선택하고, 가장 안쪽(innermost)의 은닉층들을 계산 하기 위해 동일한 가중치를 여러번 재사용 할 예정   

In [8]:
# -*- coding: utf-8 -*-
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        생성자에서 순전파 단계에서 사용할 3개의 nn.Linear 인스턴스를 생성
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D-out)
        
    def forward(self, x):
        """
        모델의 순전파 단계에서, 무작위로 0, 1, 2 또는 3 중에 하나로 선택
        은닉층을 계산하기 위해 여러번 사용한 midle_linear Model을 재사용
        
        연산 그래프를 정의할 때 동일한 Module을 여러번 재사용 해도 안전함
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random,randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred
    

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 앞에서 정의한 클래스를 생성하여 모델을 구성
model = TwoLayerNet(D_in, D_out)

# 손실 함수와 Optimizer를 만듦
# SGD 생성자에 model.parameters()를 호출하면 모델의 멤버인 2개의 nn.Linear 모듈의
# 학습 가능한 매개변수들이 포함
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상되는 y값을 계산
    y_pred = model(x)
    
    # 손실을 계산
    loss = criterion(y_pred, y)
    if t % 50 == 49:
        print(t+1, loss.item())
    
    # 변화도를 0으로 만들고, 역전파를 수행하고, 가충치를 갱신함
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

50 1.3308824300765991
100 0.007023646030575037
150 3.969906538259238e-05
200 1.9765330705467932e-07
250 9.611302864698246e-10
300 7.86831190480175e-12
350 2.7522660799722543e-12
400 2.7681769636939135e-12
450 2.3898131737420947e-12
500 2.1027066806483807e-12
